In [5]:
import pandas as pd
import yfinance as yf

ticker = "VOO"
years = 10

hist = yf.Ticker(ticker).history(period=f"{years}y", auto_adjust=True)

# Try close, else fallback to last column containing "close"
if "Close" in hist.columns:
    close = hist["Close"].dropna()
else:
    close_cols = [c for c in hist.columns if "close" in str(c).lower()]
    if not close_cols:
        raise ValueError(f"No close column found. Columns: {list(hist.columns)}")
    close = hist[close_cols[0]].dropna()

biweekly_close = close.resample("2W-FRI").last().dropna()

biweekly = pd.DataFrame({
    "close": biweekly_close,
    "chg_$": biweekly_close.diff(),
    "chg_%": biweekly_close.pct_change()
}).dropna()

print(biweekly.tail)




<bound method NDFrame.tail of                                 close      chg_$     chg_%
Date                                                      
2016-01-29 00:00:00-05:00  150.032333   4.726761  0.032530
2016-02-12 00:00:00-05:00  144.453110  -5.579224 -0.037187
2016-02-26 00:00:00-05:00  151.070572   6.617462  0.045810
2016-03-11 00:00:00-05:00  156.979004   5.908432  0.039110
2016-03-25 00:00:00-04:00  158.093094   1.114090  0.007097
...                               ...        ...       ...
2025-11-21 00:00:00-05:00  604.220032 -10.929077 -0.017767
2025-12-05 00:00:00-05:00  628.700745  24.480713  0.040516
2025-12-19 00:00:00-05:00  625.789001  -2.911743 -0.004631
2026-01-02 00:00:00-05:00  628.299988   2.510986  0.004013
2026-01-16 00:00:00-05:00  638.030029   9.730042  0.015486

[261 rows x 3 columns]>


In [6]:
# Assuming you already built `biweekly` exactly as before and it has "chg_%" column
# biweekly columns: ["close", "chg_$", "chg_%"]

threshold = -0.015  # 1.5%

total_periods = len(biweekly)
count_gt_pos = (biweekly["chg_%"] < threshold).sum()          # < -1.5%
count_gt_abs = (biweekly["chg_%"].abs() > threshold).sum()    # |chg| 

pct_gt_pos = count_gt_pos / total_periods if total_periods else float("nan")
pct_gt_abs = count_gt_abs / total_periods if total_periods else float("nan")

print(f"Total biweekly periods: {total_periods}")
print(f"Count with change < -1.5%: {count_gt_pos} ({pct_gt_pos:.2%})")
print(f"Count with |change| : {count_gt_abs} ({pct_gt_abs:.2%})")

# Optional: view the rows that triggered
hits_pos = biweekly.loc[biweekly["chg_%"] > threshold, ["close", "chg_%"]]
hits_abs = biweekly.loc[biweekly["chg_%"].abs() > threshold, ["close", "chg_%"]]

print("\nMost recent periods with change < -1.5%:")
print(hits_pos.tail(10))

print("\nMost recent periods with |change|:")
print(hits_abs.tail(10))


Total biweekly periods: 261
Count with change < -1.5%: 45 (17.24%)
Count with |change| : 261 (100.00%)

Most recent periods with change < -1.5%:
                                close     chg_%
Date                                           
2025-08-29 00:00:00-04:00  589.719788  0.002553
2025-09-12 00:00:00-04:00  601.015442  0.019154
2025-09-26 00:00:00-04:00  606.703003  0.009463
2025-10-10 00:00:00-04:00  598.815369 -0.013001
2025-10-24 00:00:00-04:00  620.793152  0.036702
2025-11-07 00:00:00-05:00  615.149109 -0.009092
2025-12-05 00:00:00-05:00  628.700745  0.040516
2025-12-19 00:00:00-05:00  625.789001 -0.004631
2026-01-02 00:00:00-05:00  628.299988  0.004013
2026-01-16 00:00:00-05:00  638.030029  0.015486

Most recent periods with |change|:
                                close     chg_%
Date                                           
2025-09-12 00:00:00-04:00  601.015442  0.019154
2025-09-26 00:00:00-04:00  606.703003  0.009463
2025-10-10 00:00:00-04:00  598.815369 -0.013001
202